In [1]:
import pandas as pd
import matplotlib.pyplot  as plt
import seaborn as sns
from simpledbf import Dbf5
import numpy as np
from tqdm import tqdm
import folium
from folium.plugins import MarkerCluster
import time
from urllib.request import urlopen
from urllib import parse
from urllib.request import Request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import json
pd.set_option('display.max_columns', 500)

## (입력) 추출지역

In [2]:
# 추출 지역 입력
Juso='세종'
# 우체국 근방 거리
dist=1

In [3]:
post = pd.read_csv('E:/post/picmt0060.csv', encoding='UTF-8', sep=",", error_bad_lines=False)
subpost=post[post['POADDR'].str.contains(Juso)]
subpost = subpost.iloc[:,[0,16,18,19,65,66]]

## 우체국 추출

In [4]:
subpost

,REGIPOCD,PONM,ZIPCD,POADDR,RDGISX,RDGISY
0,10480,정부서울청사우체국,3171.0,서울특별시 종로구 세종대로 209 (세종로),126.975222,37.574914
91,89994,우정사업본부,30114.0,"세종특별자치시 도움5로 19 (어진동,우정사업본부)",127.259285,36.504760
782,40353,여주우체국,12627.0,경기도 여주시 세종로 100,127.634153,37.289304
848,15289,세종대학교우편취급국,5006.0,"서울특별시 광진구 능동로 209 (군자동,세종대학교)",127.073306,37.550910
1795,31229,세종전의우체국,30004.0,세종특별자치시 전의면 운주산로 1208-1,127.202835,36.679975
1796,31210,세종금남우체국,30083.0,세종특별자치시 금남면 용포로 56,127.280964,36.463629
1797,35647,세종한솔동우체국,30130.0,세종특별자치시 노을3로 93,127.254350,36.479652
1798,31774,세종부강우체국,30075.0,세종특별자치시 부강면 부촌길 42,127.367070,36.529964
1799,30839,장군우체국,30053.0,세종특별자치시 장군면 장척로 408-4,127.206048,36.496969
1800,35683,정부세종청사우체국,30103.0,"세종특별자치시 도움6로 11 (어진동, 국토교통부)",127.262601,36.504920


## 우체국 기호 입력

In [5]:
REGIPOCD=30179

In [6]:
hospital = pd.read_csv('C:/Users/LeeChul-Ghi/Project/VOC_sample/data/hospital_juso.csv', encoding='UTF-8', sep=",", error_bad_lines=False)
subhospital=hospital[hospital['시도코드명'].str.contains(Juso)]
subhospital = subhospital.iloc[:,[1,3,5,6,7,8,10,27,28]]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,4,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## 병원 추출

In [7]:
subhospital=subhospital.dropna(axis=0,how='any',subset=['x좌표'])
subhospital

,요양기관명,종별코드명,시도코드명,시군구코드,시군구코드명,읍면동,주소,x좌표,y좌표
354,의료법인 영제 의료재단 엔케이세종병원,종합병원,세종시,410000,세종시,NaN,세종특별자치시 한누리대로 161 세종시 NK리움힐 상가 1~8층 (나성동),127.262248,36.479637
360,세종충남대학교병원,종합병원,세종시,410000,세종시,NaN,세종특별자치시 보듬7로 20 세종충남대학교병원 (도담동),127.257943,36.519513
1649,트리니움여성병원,병원,세종시,410000,세종시,NaN,세종특별자치시 한누리대로 1934 메인타워 (반곡동),127.303523,36.491587
1862,의료법인 선은의료재단 중앙요양병원,요양병원,세종시,410000,세종시,NaN,세종특별자치시 조치원읍 섭골길 37 (),127.288342,36.613385
1978,조치원노인전문병원,요양병원,세종시,410000,세종시,연서면,세종특별자치시 연서면 도신고복로 789-17 789,127.237865,36.613019
...,...,...,...,...,...,...,...,...,...
74161,한솔한의원,한의원,세종시,410000,세종시,NaN,"세종특별자치시 누리로 54 202호,203호,204-1호 (한솔동, 첫마을5단지)",127.251020,36.477061
74295,세종맘한의원,한의원,세종시,410000,세종시,NaN,세종특별자치시 한누리대로 265 명동프라자 6층 (나성동),127.249661,36.511359
74373,자인한의원,한의원,세종시,410000,세종시,NaN,"세종특별자치시 새롬중앙로 63 세종온누리타워 202호,203호 (새롬동)",127.250253,36.486535
74411,세종튼튼한의원,한의원,세종시,410000,세종시,NaN,세종특별자치시 한누리대로 2257 판타지타워 205호 (대평동),127.278005,36.472617


In [8]:
pharmacy = pd.read_csv('C:/Users/LeeChul-Ghi/Project/VOC_sample/data/pharmacy_juso.csv', encoding='UTF-8', sep=",", error_bad_lines=False)
subpharmacy=pharmacy[pharmacy['시도코드명'].str.contains(Juso)]
subpharmacy = subpharmacy.iloc[:,[1,3,5,6,7,8,10,13,14]]

## 약국 추출

In [9]:
subpharmacy=subpharmacy.dropna(axis=0,how='any',subset=['x좌표'])
subpharmacy

,요양기관명,종별코드명,시도코드명,시군구코드,시군구코드명,읍면동,주소,x좌표,y좌표
7,엘약국,약국,세종시,410000,세종시,NaN,"세종특별자치시 새롬중앙로 55 103호 (새롬동, 행운빌딩)",127.250338,36.485752
400,데일리약국,약국,세종시,410000,세종시,NaN,"세종특별자치시 도움1로 106 5층 516호 (종촌동, 메가시티)",127.248124,36.502816
855,성모약국,약국,세종시,410000,세종시,NaN,세종특별자치시 조치원읍 조치원2길 5 (),127.297588,36.601021
1084,다정온누리약국,약국,세종시,410000,세종시,다정동,세종특별자치시 다정중앙로 45 다정프라자 1층 110호 (다정동),127.248600,36.494441
1108,장기약국,약국,세종시,410000,세종시,장군면,세종특별자치시 장군면 장척로 405-2 (장군면),127.205549,36.497449
...,...,...,...,...,...,...,...,...,...
21900,서울약국,약국,세종시,410000,세종시,반곡동,세종특별자치시 남세종로 450 보람빌딩 104호 (보람동),127.290104,36.477407
22068,메가약국,약국,세종시,410000,세종시,NaN,"세종특별자치시 도움1로 106 4층 415호 (종촌동, 메가시티)",127.248124,36.502816
22459,건강한약국,약국,세종시,410000,세종시,한솔동,"세종특별자치시 노을3로 29 102호 (한솔동, 세종프라자)",127.259217,36.480512
22784,샬롬약국,약국,세종시,410000,세종시,조치원읍,세종특별자치시 조치원읍 새내로 72 (조치원읍),127.298800,36.598250


In [10]:
culture = pd.read_csv('C:/Users/LeeChul-Ghi/Project/VOC_sample/data/culture_juso.csv', encoding='UTF-8', sep=",", error_bad_lines=False)
subculture=culture[culture['관리기관명'].str.contains(Juso)]
subculture = subculture.iloc[:,[0,1,2,10,18,19,20,24,25]]

## 문화시설 추출

In [11]:
subculture=subculture.dropna(axis=0,how='any',subset=['위도'])
subculture

,개방시설명,개방장소명,개방시설유형구분,사용료,소재지도로명주소,소재지지번주소,관리기관명,위도,경도
3994,소담동 복합커뮤니티센터,소담동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 한누리대로 2023,NaN,세종특별자치시,36.484074,127.300370
4012,도담동 복합커뮤니티센터,도담동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 보람로 77,NaN,세종특별자치시,36.515407,127.261747
4013,종촌동 복합커뮤니티센터,종촌동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 도움3로 125,NaN,세종특별자치시,36.504635,127.246869
4014,고운동 복합커뮤니티센터,고운동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 마음로 284,NaN,세종특별자치시,36.519724,127.236588
4015,보람동 복합커뮤니티센터,보람동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 호려울로 42,NaN,세종특별자치시,36.478775,127.290614
4016,새롬동 복합커뮤니티센터,새롬동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 새롬동 341-72,NaN,세종특별자치시,36.484678,127.252503
4017,대평동 복합커뮤니티센터,대평동 복합커뮤니티센터,다목적실,NaN,세종특별자치시 대평1길 10,NaN,세종특별자치시,36.470962,127.279200
4041,세종시민체육관,세종시민체육관,체육관,NaN,세종특별자치시 조치원읍 대첩로 116,NaN,세종특별자치시,36.595189,127.284376
4042,농어민문화체육관,농어민문화체육관,체육관,NaN,세종특별자치시 연서면 월하천로 301,NaN,세종특별자치시,36.572286,127.284480
4043,침산리 족구장,침산리 족구장,체육관,NaN,세종특별자치시 조치원읍 군청로 93,NaN,세종특별자치시,36.593145,127.293376


## 행정기관 추출

In [12]:
dbf = Dbf5('C:/Users/LeeChul-Ghi/Project/VOC_sample/data/public_juso.dbf',codec='ansi')
public = dbf.to_dataframe()
subpublic=public[public['도로명주소'].str.contains(Juso)]
# subpublic = subpublic.iloc[:,[0,1,2,10,18,19,20,24,25]]

In [13]:
#naver map api key
client_id = 'xhbfckkyg3';    # 본인이 할당받은 ID 입력
client_pw = 'lpd28lbrhk1WqcLPSmEzUUSAL0mrM5lwyMCAZoaz';    # 본인이 할당받은 Secret 입력

api_url = 'https://naveropenapi.apigw.gov-ntruss.com/map-geocode/v2/geocode?query='


# 네이버 지도 API 이용해서 위경도 찾기
geo_coordi = []
for add in tqdm(subpublic['도로명주소']):
    add_urlenc = parse.quote(add)  
    url = api_url + add_urlenc
    request = Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_pw)
    try:
        response = urlopen(request)
    except HTTPError as e:
        print('HTTP Error!')
        latitude = None
        longitude = None
    else:
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)   # json
            if response_body['addresses'] != []:
                longitude = response_body['addresses'][0]['x']
                latitude = response_body['addresses'][0]['y']
#                 print("Success!",end=' ')
            elif response_body['addresses'] == []:
                latitude = None
                longitude = None
#                 print("'result' not exist!",end=' ')
            else:
                print("'addresses' not exist!",end=' ')                
        else:
#             print('Response error code : %d' % rescode)
            latitude = None
            longitude = None

    geo_coordi.append([longitude,latitude])

100%|████████████████████████████████████████████████████████████████████████████████| 180/180 [00:26<00:00,  6.86it/s]


In [14]:
subpublic

,유형,상세분류,시군구코드,도로명코드,도로명주소,기관명,위치X,위치Y,전화번호
8,경찰서,지방청,36110,2000002,세종특별자치시 한누리대로 1978 (소담동),세종특별자치시지방경찰청,9.822279e+05,1.832266e+06,182
100,경찰서,경찰서,36110,3258001,세종특별자치시 조치원읍 군청로 36,세종경찰서,9.816552e+05,1.843397e+06,182
141,경찰서,경찰서,41670,3214032,경기도 여주시 세종로 50 (창동),여주경찰서,1.012012e+06,1.921593e+06,182
295,경찰서,파출소,11140,2005001,서울특별시 중구 세종대로 141 (태평로1가),세종로파출소,9.537628e+05,1.952304e+06,02-3701-4512
580,경찰서,파출소,36110,3258069,세종특별자치시 연동면 청연로 606-1,연동파출소,9.843153e+05,1.840107e+06,044-864-7112
...,...,...,...,...,...,...,...,...,...
19423,학교,초등학교,36110,4574526,세종특별자치시 전동면 하노장3길 9,전동초등학교,9.790906e+05,1.851029e+06,044-863-1208
19424,학교,초등학교,36110,4574530,세종특별자치시 소정면 학교말길 10,소정초등학교,9.694921e+05,1.858120e+06,044-566-6060
20675,학교,초등학교,41670,3214032,경기도 여주시 세종로 204-19 (교동),세종초등학교,1.011864e+06,1.920105e+06,031-880-2600
20677,학교,초등학교,41670,3214032,경기도 여주시 세종로 488-4 (점봉동),점봉초등학교,1.012826e+06,1.917549e+06,031-881-6604


In [16]:
np_geo_coordi = np.array(geo_coordi)
subpublic = pd.DataFrame({"상세분류": subpublic['상세분류'].values,
                              "기관명": subpublic['기관명'].values,
                              "경도": np_geo_coordi[:, 0],
                              "위도": np_geo_coordi[:, 1]})
subpublic=subpublic.dropna(axis=0,how='any',subset=['위도'])
subpublic['경도']=pd.to_numeric(subpublic['경도'])
subpublic['위도']=pd.to_numeric(subpublic['위도'])
subpublic

,상세분류,기관명,경도,위도
0,지방청,세종특별자치시지방경찰청,127.301576,36.487881
1,경찰서,세종경찰서,127.294887,36.588247
2,경찰서,여주경찰서,127.634723,37.293695
3,파출소,세종로파출소,126.976447,37.568950
4,파출소,연동파출소,127.324870,36.558576
...,...,...,...,...
175,초등학교,전동초등학교,127.265670,36.656585
176,초등학교,소정초등학교,127.159011,36.721131
177,초등학교,세종초등학교,127.633771,37.279894
178,초등학교,점봉초등학교,127.644792,37.257191


In [17]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [18]:
df=subpost[subpost.REGIPOCD==REGIPOCD]

In [19]:
df

,REGIPOCD,PONM,ZIPCD,POADDR,RDGISX,RDGISY
3040,30179,세종우체국,30151.0,세종특별자치시 시청대로 180 (보람동),127.291879,36.480534


In [20]:
for i in df.index:
    lat = df.loc[i,'RDGISY']
    long = df.loc[i,'RDGISX']
    title = df.loc[i,'PONM']
m = folium.Map([lat,long],zoom_start=15)
folium.Marker([lat,long],icon=folium.Icon(color='orange', icon='home', prefix='fa'),tooltip=title).add_to(m)
folium.Circle([lat,long],tooltip=title,radius=dist*1000).add_to(m)

In [21]:
res=[]
x=[]
#'beige', 'lightgray', 'darkgreen', 'cadetblue', 'gray', 'orange', 'blue', 'darkblue', 'lightgreen', 'darkpurple', 'lightblue', 'black', 'darkred', 'white', 'red', 'purple', 'lightred', 'pink', 'green'
for i in subhospital.index:
    sub_lat =  subhospital.loc[i,'y좌표']
    sub_long = subhospital.loc[i,'x좌표']
    title = subhospital.loc[i,'요양기관명']
    if GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat)<dist:
        folium.Marker([sub_lat,sub_long],icon=folium.Icon(color='red', icon='plus'),tooltip=title).add_to(m)
        print(title)
for i in subpharmacy.index:
    sub_lat =  subpharmacy.loc[i,'y좌표']
    sub_long = subpharmacy.loc[i,'x좌표']
    title = subpharmacy.loc[i,'요양기관명']
    if GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat)<dist:
        folium.Marker([sub_lat,sub_long],icon=folium.Icon(color='cadetblue', icon='plus-sign'),tooltip=title).add_to(m)
        print(title)
for i in subculture.index:
    sub_lat =  subculture.loc[i,'위도']
    sub_long = subculture.loc[i,'경도']
    title = subculture.loc[i,'개방장소명']
    if GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat)<dist:
        folium.Marker([sub_lat,sub_long],icon=folium.Icon(color='green', icon='bell'),tooltip=title).add_to(m)
        print(title)
for i in subpublic.index:
    sub_lat =  subpublic.loc[i,'위도']
    sub_long = subpublic.loc[i,'경도']
    title = subpublic.loc[i,'기관명']
    if GeoUtil.get_harversion_distance(sub_long, sub_lat,long, lat)<dist:
        folium.Marker([sub_lat,sub_long],icon=folium.Icon(color='purple', icon='info-sign'),tooltip=title).add_to(m)        
        print(title)
m

세종성모소아청소년과의원
코알이비인후과의원
소담내과의원
무지개소아청소년과의원
우리연합의원
드림이비인후과의원
소담튼튼소아청소년과의원
고운소아청소년과의원
정마취통증의학과의원
코끼리소아청소년과의원
아름누리소아청소년과의원
소망안과의원
세종고려정형외과의원
더공감정신건강의학과의원
세종탑재활의학과의원
이도내과의원
노블클라쎄산부인과의원
센텀비뇨기과의원
세종시청점오라클피부과의원
세종미소치과의원
세종보람치과의원
서울소담치과의원
연세감동치과의원
김만주정성치과의원
세종드림치과의원
장효길한의원
세종소담한의원
봄뜰부부한의원
청심당한의원
경희바른몸한의원
본한의원
동인한의원
생생당한의원
태이재한의원
미소약국
메디팜큰사랑약국
병아리약국
본약국
보람약국
세종시티약국
늘봄약국
쓰담약국
소담약국
새샘약국
서울약국
소담동 복합커뮤니티센터
보람동 복합커뮤니티센터
보람지구대
세종우체국
세종특별자치시청
소담동주민센터
보람동주민센터
보람고등학교
세종장영실고등학교
소담중학교
보람중학교
글벗초등학교
소담초등학교
보람초등학교
여울초등학교
금남초등학교


In [22]:
m.save('C:/Users/LeeChul-Ghi/Project/VOC_sample/sejong_sample.html')